In [1]:
# data
from total_export_data import *

# data management
import pandas as pd
import numpy as np
import networkx as nx
import operator
import matplotlib.pyplot as plt
from networkx.algorithms.centrality import eigenvector_centrality
from networkx.algorithms.centrality import degree_centrality
from networkx.algorithms.cluster import clustering
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community import greedy_modularity_communities
from networkx import edge_betweenness_centrality as betweenness

#Visualization
import plotly.plotly as py
import plotly.graph_objs as go

# Tools
import itertools

In [2]:
file = 'downloads/country_partner_hsproduct4digit_year_2016.csv'
df_final = total_export(file)

/Users/phuongpham/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning:

Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.



>>> Total export == total import ... True


In [4]:
all_edges = df_final[['location_code','partner_code', 'export_portion']].values

In [5]:
## Build a graph with directed edges going from country which exports to country which imports
network = nx.DiGraph()

In [6]:
network.add_weighted_edges_from(all_edges)

In [7]:
# Degree centrality
degrees_cen = degree_centrality(network)

In [8]:
# Eigenvector centrality
eigenvector = eigenvector_centrality(network, weight = 'weight', max_iter = 1000)

In [9]:
# Position
pos = nx.random_layout(network, seed = 1)


In [10]:
def plot_node_edge(network, attribute, attribute_name, scale = 10):
    
    # Choose a top 5% of edges to add:
    cutoff = np.quantile(df_final['export_portion'],1-5/100)
    add_edges = [edge for edge in network.edges(data=True)
       if edge[2]['weight'] > cutoff]
    add_nodes = set()
    
    
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5,color='#888'),
        opacity = 0.3,
        hoverinfo='none',
        mode='lines')
    
    for edge in add_edges:
        start, dest,_ = edge
        add_nodes.add(start)
        add_nodes.add(dest)
        if dest == 'ANS' or start == 'ANS': #Undeclared
            continue 
        x0, y0 = pos[start]
        x1, y1 = pos[dest]
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])
        
    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        textposition = 'top center',
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='Jet',
            reversescale=True,
            color=[],
            size=[],
            colorbar=dict(
                thickness=15,
                title=attribute_name,
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))
    
    for node in add_nodes:
        if node == 'ANS': #Undeclared
            continue
        x, y = pos[node]
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['marker']['size']+= tuple([attribute[node]*scale])
        node_trace['marker']['color']+=tuple([attribute[node]])
        node_trace['text']+=tuple([node])
    return edge_trace, node_trace

In [25]:
# Eigenvector centrality scale 100
# Degree centrality scale 10
edge_trace, node_trace = plot_node_edge(network, eigenvector, 'Eigenvector centrality', scale = 100)

In [26]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>International Trade Network in 2016',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')

In [94]:
# Finding trade community
# k = 3
# comp = girvan_newman(network)
# for communities in itertools.islice(comp, k):
#     print(tuple(sorted(c) for c in communities))

KeyboardInterrupt: 

In [83]:
len(c[0])

232